In [23]:
import re
import glob
import pandas as pd
import sweetviz
import random
import pandas as pd
import warnings
import unidecode
import numpy as np
import csv
import emoji
from typing import List
from flair.data import Token
import sys,os
import tqdm
sys.path.append('../')

In [6]:
from preprocessing.amharicSegmenter import AmharicSegmenter

In [11]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [13]:
def remove_url(data):
    return data.replace(find_url(data),"")

In [14]:
def remove_emoji(news):
    for idx,cnt in enumerate(news):
        news[idx] = re.sub(r'(:\S+)(@\w+)[A-Za-z]','', cnt)
    return news

In [15]:
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

In [18]:
remove = ['\xa087','\n','\xa0', '|', '/','፤','…','፣','«','(',')','"','“','”','»','-','.',',',
          '@','#','$','%','^','&','*','[',']','{','}',';','›','’','‘','‹','<','>','`','´','~','=','+','፦','፥','፧','፠']
May_news_DF = pd.DataFrame()
#May_news  = glob.glob("/srv/data/amharic/crawl/output/*.csv")
#May_news  = glob.glob("/srv/data/amharic/crawl/output/2020-1-28-0-0-3.csv")
May_news  = glob.glob("data/*.csv")
for news in May_news:
    data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"] ,encoding="utf-8")
    #Removing duplicated sentences from Content
    data = data[data.Content.duplicated()==False].reset_index()
   
    for token in remove:
        data.Content = data.Content.apply(lambda x: x.replace(token,' ')) 
    #Removing the url from Content
    data.Content = data.Content.apply(lambda x: remove_url(x))
    data = data.drop(['index','ID','URL','Date','Media'], axis=1)
    #Merging the Dataframes
    May_news_DF = pd.concat([May_news_DF,data])

print("Before Removing Duplicated Sentences")
print(duplicated_values_data(May_news_DF))

May_news_DF = May_news_DF[May_news_DF.Content.duplicated()==False].reset_index()
May_news_DF = May_news_DF.drop(['index'], axis=1)
print("="*40,"\nAfter Removing Duplicated Sentences")
print(duplicated_values_data(May_news_DF))


Before Removing Duplicated Sentences
   Columns  Duplicate count
0  Content              280
After Removing Duplicated Sentences
   Columns  Duplicate count
0  Content                0


In [25]:
May_news_DF.Content.to_csv("MayNews_Content.csv", index=False)
df = pd.read_csv("MayNews_Content.csv")

# Amharic Text Segmentation (sentence and tokens)

In [26]:
sent_punct = []
word_punct = []
amseg = AmharicSegmenter(sent_punct,word_punct)
#text=pd.DataFrame(df)
text = df.to_string(index = False) 
#text  =  "በዚህም መሠረት የአማራ:: ዴሞክራሲያዊ ኃይል ንቅናቄ የመረጠው: ምልክት ከዚህ በፊት በነበረ፡፡ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው፡፡ የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የጋሞ ዴሞክራሲያዊ ፓርቲ የመረጠው ምልክት ከዚህ በፊት በነበረ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የኢትዮጵያ ዜጎች ለማኅበራዊ ፍትሕ (ኢዜማ) የመረጠው ምልክት ከዚህ በፊት በነበረ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የምዕራብ ሶማሌ ዴሞክራሲያዊ ፓርቲ ከሌላ ፓርቲ ምልክት ጋር ምልክቱ የተቀራረበ በመሆኑ፣ እንዲሁም የአርጎባ ብሔረሰብ ዴሞክራሲያዊ ንቅናቄ ከሌላ ፓርቲ ምልክት ጋር ምልክቱ የተቀራረበ በመሆኑ ነው እንዲቀይሩ በምርጫ ቦርድ ማሳሰቢያ የተሰጣቸው፡፡"
with open ("all_sentences.txt","w", encoding="utf-8") as all_sentences:
  for s in amseg.tokenize_sentence(text):
      all_sentences.write(s+'\n')

In [ ]:
text  =  "በዚህም መሠረት የአማራ:: ዴሞክራሲያዊ ኃይል ንቅናቄ የመረጠው: ምልክት ከዚህ በፊት በነበረ፡፡ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው፡፡ የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የጋሞ ዴሞክራሲያዊ ፓርቲ የመረጠው ምልክት ከዚህ በፊት በነበረ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የኢትዮጵያ ዜጎች ለማኅበራዊ ፍትሕ (ኢዜማ) የመረጠው ምልክት ከዚህ በፊት በነበረ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የምዕራብ ሶማሌ ዴሞክራሲያዊ ፓርቲ ከሌላ ፓርቲ ምልክት ጋር ምልክቱ የተቀራረበ በመሆኑ፣ እንዲሁም የአርጎባ ብሔረሰብ ዴሞክራሲያዊ ንቅናቄ ከሌላ ፓርቲ ምልክት ጋር ምልክቱ የተቀራረበ በመሆኑ ነው እንዲቀይሩ በምርጫ ቦርድ ማሳሰቢያ የተሰጣቸው፡፡"
#with open ("all_tokens.txt","w", encoding="utf-8") as all_tokens:
amseg = AmharicSegmenter(sent_punct,word_punct)
amseg.amharic_tokenizer(text)
  

# Read each CSV files and write it to text file

In [10]:
# remove output file if it exists, then append each csv to a single file output
outfile = "all_sentences.txt"
os.remove(outfile) if os.path.exists(outfile) else None
# all csv files, to controll progress par
allnews = []
for news in glob.glob("data/*.csv"):
    allnews.append(news)

# read each files and write to a file system   
for news in tqdm.tqdm(allnews,  position=0, leave=True):
    data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"] ,encoding="utf-8")
    data = data[data.Content.duplicated()==False].reset_index()
    text  = data.Content.to_string()
    with open (outfile,"a", encoding="utf-8") as all_sentences:
        for s in amseg.tokenize_sentence(text):
            all_sentences.write(s+'\n')


100%|██████████| 4/4 [00:05<00:00,  1.41s/it]
